# Evaluating Fairness in Machine Learning: Comparative Analysis and Benchmarking of Fairlearn and AIF360 | COMPAS Recidivism Dataset

In [8]:
%load_ext autoreload
%autoreload 2

import os
import random

import pandas as pd
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

from src.constants import RANDOM_STATE, DEFAULT_MODEL_CONFIG
from src.evaluation import calculate_metrics, plot_roc, print_confusion_matrix

In [9]:
sklearn.set_config(transform_output="pandas")

## Ensure reproducibility

Set random seeds for reproducibility.

In [10]:
np.random.seed(RANDOM_STATE)
os.environ["PYTHONHASHSEED"] = str(RANDOM_STATE)
random.seed(RANDOM_STATE)

## Load data

In [16]:
FEATURES = ["sex",
            "age",
            "age_cat",
            "race",
            "juv_fel_count",
            "juv_misd_count",
            "juv_other_count",
            "priors_count",
            "days_b_screening_arrest",
            "c_days_from_compas",
            "c_charge_degree",
            "decile_score.1",
            "score_text",
            "v_type_of_assessment",
            "v_decile_score",
            "v_score_text",
            "end",
            ]
TARGET = "is_recid"
PROTECTED_ATTRIBUTE = "race"

In [17]:
data = pd.read_csv("../data/compas-scores-two-years.csv", usecols=[*FEATURES, TARGET])

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7214 entries, 0 to 7213
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sex                      7214 non-null   object 
 1   age                      7214 non-null   int64  
 2   age_cat                  7214 non-null   object 
 3   race                     7214 non-null   object 
 4   juv_fel_count            7214 non-null   int64  
 5   juv_misd_count           7214 non-null   int64  
 6   juv_other_count          7214 non-null   int64  
 7   priors_count             7214 non-null   int64  
 8   days_b_screening_arrest  6907 non-null   float64
 9   c_days_from_compas       7192 non-null   float64
 10  c_charge_degree          7214 non-null   object 
 11  is_recid                 7214 non-null   int64  
 12  decile_score.1           7214 non-null   int64  
 13  score_text               7214 non-null   object 
 14  v_type_of_assessment    

In [19]:
y = data[TARGET]
z = data[PROTECTED_ATTRIBUTE]
X = data.drop(columns=TARGET)

In [20]:
X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(X, y, z, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

In [21]:
MODEL_CONFIG = dict(DEFAULT_MODEL_CONFIG, cat_features=X.select_dtypes("object").columns.to_list())

In [22]:
default_model = CatBoostClassifier(**MODEL_CONFIG)
model = default_model.copy()

In [23]:
model.fit(X_train, y_train)

0:	learn: 0.6844627	total: 58.5ms	remaining: 2m 55s
250:	learn: 0.3208956	total: 711ms	remaining: 7.78s
500:	learn: 0.3091280	total: 1.78s	remaining: 8.89s
750:	learn: 0.3038870	total: 2.79s	remaining: 8.37s
1000:	learn: 0.2994238	total: 3.84s	remaining: 7.67s
1250:	learn: 0.2951383	total: 5s	remaining: 6.99s
1500:	learn: 0.2916133	total: 6.01s	remaining: 6s
1750:	learn: 0.2886633	total: 6.67s	remaining: 4.76s
2000:	learn: 0.2859524	total: 7.34s	remaining: 3.67s
2250:	learn: 0.2837542	total: 8.13s	remaining: 2.7s
2500:	learn: 0.2815680	total: 8.8s	remaining: 1.76s
2750:	learn: 0.2795766	total: 9.75s	remaining: 882ms
2999:	learn: 0.2775952	total: 10.5s	remaining: 0us


In [0]:
y_pred = pd.Series(model.predict(X_test), name="y_pred", index=y_test.index)

In [ ]:
calculate_metrics(model, X_test, y_test)

## Fairness assessment

### Fairlearn

#### Detection

In [16]:
from fairlearn.metrics import demographic_parity_ratio, demographic_parity_difference

In [17]:
demographic_parity_difference(y_test, y_pred, sensitive_features=z_test, method="between_groups")

0.30000000000000004

In [18]:
demographic_parity_difference(y_test, y_pred, sensitive_features=z_test, method="to_overall")

0.22999999999999998

In [19]:
demographic_parity_ratio(y_test, y_pred, sensitive_features=z_test)

0.625

#### Mitigation

In [20]:
from fairlearn.reductions import DemographicParity, ExponentiatedGradient

In [22]:
reduction = ExponentiatedGradient(estimator=default_model.copy(), constraints=DemographicParity(difference_bound=0.01), max_iter=10)
reduction.fit(X_train, y_train, sensitive_features=z_train)

/opt/homebrew/Caskroom/miniconda/base/envs/fairness-toolkits/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/opt/homebrew/Caskr

0:	learn: 0.6899831	total: 530us	remaining: 1.59s
250:	learn: 0.4698964	total: 102ms	remaining: 1.12s
500:	learn: 0.4234232	total: 198ms	remaining: 989ms
750:	learn: 0.3885434	total: 302ms	remaining: 906ms
1000:	learn: 0.3550001	total: 401ms	remaining: 802ms
1250:	learn: 0.3265120	total: 500ms	remaining: 699ms
1500:	learn: 0.3027580	total: 599ms	remaining: 599ms
1750:	learn: 0.2828691	total: 699ms	remaining: 499ms
2000:	learn: 0.2648067	total: 798ms	remaining: 398ms
2250:	learn: 0.2487316	total: 894ms	remaining: 297ms
2500:	learn: 0.2338165	total: 995ms	remaining: 199ms
2750:	learn: 0.2203859	total: 1.09s	remaining: 99ms
2999:	learn: 0.2079273	total: 1.19s	remaining: 0us
0:	learn: 0.6899831	total: 542us	remaining: 1.63s
250:	learn: 0.4698964	total: 98.2ms	remaining: 1.07s
500:	learn: 0.4234232	total: 197ms	remaining: 984ms
750:	learn: 0.3885434	total: 300ms	remaining: 897ms
1000:	learn: 0.3550001	total: 397ms	remaining: 792ms
1250:	learn: 0.3265120	total: 496ms	remaining: 693ms
1500:	l

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.DemographicParity object at 0x156728d90>,
                      estimator=<catboost.core.CatBoostClassifier object at 0x156749f30>,
                      max_iter=10, nu=0.0038551689859054623)

In [23]:
y_pred_reduced = reduction.predict(X_test)

In [24]:
calculate_metrics(reduction, X_test, y_test)

{'Accuracy': 0.74,
 'Precision': 0.7894736842105263,
 'Recall': 0.8571428571428571,
 'F1 Score': 0.821917808219178}

In [25]:
demographic_parity_difference(y_test, y_pred_reduced, sensitive_features=z_test, method="to_overall")

0.08333333333333337

In [26]:
demographic_parity_ratio(y_test, y_pred_reduced, sensitive_features=z_test, method="to_overall")

0.8888888888888888

### AIF360

In [27]:
from aif360.datasets import BinaryLabelDataset

In [50]:
dataset_aif360_train = BinaryLabelDataset(df=pd.concat([X_train, y_train], axis=1), 
                                    label_names=[TARGET],
                                    protected_attribute_names=[PROTECTED_ATTRIBUTE],
                                    )

dataset_aif360_test = BinaryLabelDataset(df=pd.concat([X_test, y_test], axis=1), 
                                    label_names=[TARGET],
                                    protected_attribute_names=[PROTECTED_ATTRIBUTE],
                                    )

predictions_aif360 = BinaryLabelDataset(df=X_test.assign(**{TARGET: y_pred}), 
                                        label_names=[TARGET],
                                        protected_attribute_names=[PROTECTED_ATTRIBUTE],
                                        )

#### Detection

_"Since the main computation of confusion matrices is common for a large set of metrics, we utilize memoization and caching of computations for performance on large-scale datasets."_

In [29]:
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.explainers import MetricTextExplainer

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[inFairness]'


In [30]:
priv = [{PROTECTED_ATTRIBUTE: 1}, {PROTECTED_ATTRIBUTE: 3}, {PROTECTED_ATTRIBUTE: 4}]
unpriv = [{PROTECTED_ATTRIBUTE: 2}]

In [31]:
cm = ClassificationMetric(dataset_aif360_test,
                         predictions_aif360,
                         privileged_groups=priv,
                         unprivileged_groups=unpriv,
                         )

In [32]:
cm.disparate_impact()

0.8439955106621774

In [33]:
cm.statistical_parity_difference()

-0.12065972222222221

In [34]:
text_expl = MetricTextExplainer(cm)

In [35]:
text_expl.statistical_parity_difference()

'Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.12065972222222221'

In [36]:
from aif360.sklearn.metrics import statistical_parity_difference, disparate_impact_ratio

pip install 'aif360[OptimalTransport]'


In [37]:
statistical_parity_difference(y_test, y_pred, prot_attr=z_test, priv_group=3)

-0.1473684210526316

In [38]:
disparate_impact_ratio(y_test, y_pred, prot_attr=z_test, priv_group=3)

0.8157894736842105

___

In [39]:
pd.concat([y_test, y_pred], axis=1).groupby(z_test, sort=False).sum().div(z_test.value_counts(), axis=0)

,Creditability,y_pred
Sex & Marital Status,,
1,0.666667,0.500000
2,0.625000,0.652778
3,0.761905,0.800000
4,0.647059,0.705882


In [40]:
pd.concat([y_test, y_pred], axis=1).mean()

Creditability    0.70
y_pred           0.73
dtype: float64

In [41]:
y_pred.loc[z_test.ne(3)].mean() - y_pred.loc[z_test.eq(3)].mean() 

-0.1473684210526316

#### Mitigation

In [52]:
from aif360.algorithms.inprocessing import ExponentiatedGradientReduction as ExponentiatedGradientReductionAif

In [56]:
reduction_aif = ExponentiatedGradientReductionAif(estimator=default_model.copy(), constraints=DemographicParity(difference_bound=0.01), max_iter=10)

In [57]:
reduction_aif.fit(dataset_aif360_train)

/opt/homebrew/Caskroom/miniconda/base/envs/fairness-toolkits/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniconda/base/envs/fairness-toolkits/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer

0:	learn: 0.6896976	total: 513us	remaining: 1.54s
250:	learn: 0.4713271	total: 96.1ms	remaining: 1.05s
500:	learn: 0.4265194	total: 199ms	remaining: 991ms
750:	learn: 0.3916272	total: 301ms	remaining: 901ms
1000:	learn: 0.3590966	total: 403ms	remaining: 805ms
1250:	learn: 0.3306166	total: 503ms	remaining: 704ms
1500:	learn: 0.3073161	total: 604ms	remaining: 603ms
1750:	learn: 0.2868668	total: 704ms	remaining: 502ms
2000:	learn: 0.2696378	total: 799ms	remaining: 399ms
2250:	learn: 0.2536365	total: 895ms	remaining: 298ms
2500:	learn: 0.2391847	total: 997ms	remaining: 199ms
2750:	learn: 0.2255040	total: 1.1s	remaining: 99.7ms
2999:	learn: 0.2136827	total: 1.2s	remaining: 0us
0:	learn: 0.6896976	total: 511us	remaining: 1.53s
250:	learn: 0.4713271	total: 100ms	remaining: 1.09s
500:	learn: 0.4265194	total: 198ms	remaining: 990ms
750:	learn: 0.3916272	total: 300ms	remaining: 899ms
1000:	learn: 0.3590966	total: 402ms	remaining: 802ms
1250:	learn: 0.3306166	total: 502ms	remaining: 701ms
1500:	l

In [77]:
y_pred_reduced_aif = reduction_aif.predict(dataset_aif360_test).labels

In [78]:
statistical_parity_difference(y_test, y_pred_reduced_aif, prot_attr=z_test, priv_group=3)

-0.09573934837092724

In [80]:
disparate_impact_ratio(y_test, y_pred_reduced_aif, prot_attr=z_test, priv_group=3)

0.8788839568801523